# Temperaturausgleich


In [1]:
using MMJMesh
using MMJMesh.Plots
using MMJMesh.Meshes
using MMJMesh.Utilities
using MMJMesh.Geometries

using SparseArrays
using DifferentialEquations
using CairoMakie: Figure, Axis, scatter!, lines, scatterlines, scatterlines!, update_theme!

include("fem.jl")
include("heat.jl")
update_theme!(colormap=:acton)

## Netz

In [ ]:
m = Mesh("gmsh/thermal-equilibrium.msh")

nl = [i for i = nodeindices(m) if coordinate(node(m, i), 1) < 0.1]
nr = [i for i = nodeindices(m) if coordinate(node(m, i), 1) > 0.1]

p = mplot(m, edgesvisible=true) |> mconf()
scatter!(coordinates(m, nl), color=:hotpink)
scatter!(coordinates(m, nr), color=:steelblue)
p

## Elementfunktionen

In [3]:
setdata!(group(m, :elements), :ke_func, heat_ke(401))
setdata!(group(m, :elements), :me_func, heat_me(8960, 3850))

## Anfangswert

In [ ]:
t0 = zeros(nnodes(m))
t0[nl] .= -10
t0[nr] .= 10
mplot(m, t0) |> mconf()

## Anfangswertproblem definieren und lösen

In [ ]:
K, M, r = assemble_kmr(m)
F(T, _, _) = -K * T
JF(_, _, _) = -K
p = ODEProblem(ODEFunction(F, jac=JF, mass_matrix=M), t0, [0.0, 3600.0])
@time s = solve(p, QNDF());

## Ergebnisse darstellen

In [ ]:
t = s.t
y = stack(s.u, dims=1)
i1 = nodeindex(m, on(Point(0.0, 0.0)))
i2 = nodeindex(m, on(Point(0.2, 0.0)))
p = scatterlines(t, y[:, i1])
scatterlines!(t, y[:, i2])
p

In [ ]:
mplot(m, s[end], edgesvisible=true) |> mconf()